## Abstract

The dimensionality of representation is a critical question to solve in unsupervised learning when we try to represent objects with multiple latent features. One method to solve this question is to apply a Bayesian latent feature model where the prior is used to represent the number of latent features. The Indian buffet process is an efficient stochastic process that can generate a prior for a model to represent objects with an infinite number of latent features. In this paper, we implement the basic algorithm of a binary latent-feature model where the prior is generated by the Indian buffet process. We explore ways to optimize the algorithm and test the algorithm using simulated data sets and real data sets. We also compare the algorithm with two competing algorithms. 

## 1. Introduction

Unsupervised learning techniques have gained attention in academia and the industry due to their application in a variety of fields, such as image and pattern recognition, cancer research, consumer research, etc (Bouguila et al, 2005; ISLR). These techniques can help us identify the properties of objects even in the absence of response variables. The properties of these objects can be better captured if we use multiple latent features to represent the objects (Griffiths &  Ghahramani, 2005). As we work with latent features, one question we often need to consider is how many latent features do we need to use to represent the objects (Griffiths &  Ghahramani, 2005). The number of latent features can be finite or infinite. If we assume the dimension is infinite, one way to determine the latent structure is to use a Bayesian latent feature model, in which we use a prior distribution to represent the number of latent features needed, and use the likelihood function to analyze how these latent features are associated with the objects in the data set. The Indian buffet process provides an efficient way to generate a prior distribution for a latent feature model over equivalence classes of binary matrices with a finite number of rows but potentially an infinite number of columns. Such a model can be used to represent each object with a large number of latent features. 

One of the applications of the Indian buffet process is image processing. For example, Dang and Chainais applied the Indian buffet process as a prior and proposed a Bayesian nonparametric approach called Indian buffet process dictionary learning. The algorithm was applied to image inpainting and compressive sensing (Dang & Chainais, 2017). 

In this paper, we implement the algorithm of the Indian buffet process described in the paper titled "Infinite Latent Feature Models and the Indian Buffet Process" by Thomas L. Griffiths and Zoubin Ghahramani. We use the Indian buffet process to generate a prior for a linear-Gaussian binary latent feature model described in the paper, and implemented a Gibbs sampler to find the most frequent latent features. We explore ways to make the algorithm more efficient using computational techniques in Python and test the algorithm using simulated data and real-world data. We also compare the efficiency of our algorithm with two competing algorithms.



## 2. Description of Algorithm

### 2.1 Indian Buffet Process

We can use the Indian buffet process to generate a binary matrix $\textbf{Z}$ that shows which latent features are associated with each of the objects. If $k$th feature is associated with the $i$th object, then we will set $z_{ik} = 1$; otherwise, we will set $z_{ik} = 0$. Suppose we have N objects, then the binary matrix $\textbf{Z}$ would have a dimension of $N \times D$, where D is the number of the features that we would have at the end of the process. 

The Indian buffet process draws an analogy with Indian buffet restaurants in London. The restaurants have a large number of dishes for customers to choose from. Suppose we have N customers entering the restaurant one after another. The first customer takes the first Possison$(\alpha)$ dishes. The dishes that were taken by this customer are "recorded" in the first row of the matrix $\textbf{Z}$ by setting $z_{1k} = 1$ where $k$ represents the location of the dishes taken by this customer. All dishes that are not taken by this customer remain as $0$ in $\textbf{Z}$. Then for each of the customers followed, the $i$th customer considers all the dishes that have been taken by all previous customers based on popularity, and takes each dish with a probability of $\frac{m_k}{i}$ where $m_k$ is the number of customers who have tried the dish. Once this customer has considered all the dishes that have been tried by previous customers, s/he tries Poisson$(\alpha/i)$ dishes that have not been tried by any of the previous customers. All the dishes that are taken by each customer are "recorded" in $\textbf{Z}$ in the same manner.

The probability of getting a matrix $\textbf{Z}$ is as follows:

\begin{align*}
P(\textbf{Z}) = \frac{\alpha^{K_+}}{\prod_{i=1}^{K+} K_1^{(i)}!} exp\{-\alpha H_N\} \prod_{k=1}^{K+} \frac{(N - m_k)!(m_k - 1)!}{N!}
\end{align*}

where $K^{+}$ represents the number of dishes that have been tried by at least one customer (i.e. $m_k > 0$), and $K_1^{(i)}$ represents the number of new dishes taken by the $i$th customer.

Once we know the distribution of $\textbf{Z}$, we can define the full condtional for $z_{ik = 1}$ in $\textbf{Z}$ as $P(z_{ik} = 1 \mid \textbf{Z}_{-ik})$. But we only need to condition on the elements in the same column $\textbf{z}_{-i,k}$ since columns are independent. Therefore, we have the full conditional for $z_{ik}$ as follows:

\begin{align*}
P(z_{ik = 1} \mid \textbf{z}_{-i,k}) = \frac{m_{-i,k}}{N}
\end{align*}

### 2.2 Applying the Indian buffet process to a linear-Gaussian binary latent feature model

We use the linear-Gaussian binary latent feature model dervied in the paper, where the likelihood function is as follows:

\begin{align*}
P(\textbf{X} \mid \textbf{Z}, \sigma_X, \sigma_A) = \frac{1}{(2\pi)^{ND/2}\sigma_X^{(N - K)D} \sigma_A^{KD}
\lvert \textbf{Z}^T\textbf{Z} + \frac{\sigma_X^2}{\sigma_A^2}I \rvert ^ {D/2}}
exp\{-\frac{1}{2\sigma_X^2} tr(\textbf{X}^T(\textbf{I} - \textbf{Z}(\textbf{Z}^T \textbf{Z} + \frac{\sigma_X^2}{\sigma_A^2}I)^{-1} \textbf{Z}^T)\textbf{X}) \}
\end{align*}

To avoid potential underflow/overflow during the computation, we compute the log-likelihood function first and exponentiate it to get the likelihood. The log-likelihood function has the following form:

\begin{align*}
l(\textbf{X} \mid \textbf{Z}, \sigma_X, \sigma_A) &= -\{ \frac{ND}{2} log(2\pi) + \frac{N - K}{D} log(\sigma_X) +
KD log(\sigma_A) + \frac{D}{2}log(\lvert \textbf{Z}^T\textbf{Z} + \frac{\sigma_X^2}{\sigma_A^2}I \rvert) \} \\
&-\frac{1}{2\sigma_X^2} tr(\textbf{X}^T(\textbf{I} - \textbf{Z}(\textbf{Z}^T \textbf{Z} + \frac{\sigma_X^2}{\sigma_A^2}I)^{-1} \textbf{Z}^T)\textbf{X})
\end{align*}

With the likelihood function, we can find the full conditional of $z_{ik}$ by multiplying the likelihood function by the prior mentioned earlier:

\begin{align*}
P(z_{ik} \mid \textbf{X}, \textbf{Z}_{-ik}, \sigma_X, \sigma_A) \propto P(\textbf{X} \mid \textbf{Z}, \sigma_X, \sigma_A) P(z_{ik} \mid \textbf{z}_{-i,k}) 
\end{align*}


## Reference

Bouguila, Nizar, and Djemel Ziou. “Using Unsupervised Learning of a Finite Dirichlet Mixture Model to Improve Pattern Recognition Applications.” Pattern Recognition Letters, vol. 26, no. 12, 2005, pp. 1916–1925., doi:10.1016/j.patrec.2005.03.016. 

Dang, Hong-Phuong, and Pierre Chainais. “Indian Buffet Process Dictionary Learning: Algorithms and Applications to Image Processing.” International Journal of Approximate Reasoning, vol. 83, 2017, pp. 1–20., doi:10.1016/j.ijar.2016.12.010. 

Griffiths, Thomas L., and Zoubin Ghahramani. “Infinite Latent Feature Models and the Indian Buffet Process.” 2005. 

An Introduction to Statistical Learning: with Applications in R, by Gareth James et al., Springer, 2021, pp. 373–374. 